In [ ]:
#!pip install transformers accelerate

from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
import torch

# Define Anupama Pandey's data directly in code
anupama_data = """
Anupama Pandey is my beloved mother who brings joy to our family.
She was born on December 4th and we celebrate her birthday with great enthusiasm.
Anupama hails from Lucknow, Uttar Pradesh, a city known for its rich cultural heritage.
She works at the Civil Court where she serves with dedication and integrity.
With a Master's degree in Ancient History, she has deep knowledge about historical civilizations.
Her green thumb makes our garden flourish with roses, marigolds and tulips.
In the kitchen, she creates magic - her biryani and gulab jamun are legendary.
During leisure time, she enjoys watching Instagram reels about gardening tips and cooking hacks.
Her favorite historical era is the Mughal period, especially Akbar's reign.
Anupama loves sharing stories about Lucknow's architectural wonders from her childhood.
"""

# Load mobile-friendly model (TinyStories 33M parameters)
#model = GPT2LMHeadModel.from_pretrained("roneneldan/TinyStories-33M")
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

from transformers import GPT2LMHeadModel, GPT2Tokenizer
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

tokenizer.pad_token = tokenizer.eos_token


# Prepare dataset directly in memory
formatted_data = anupama_data.replace(". ", ".\n")
tokenized_data = tokenizer(
    formatted_data,
    return_tensors='pt',
    max_length=128,
    truncation=True,
    padding='max_length'
)

# Create dataset class for in-memory training
class InMemoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = InMemoryDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training setup
training_args = TrainingArguments(
    output_dir="./anupama_model",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=3e-4,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# Start fine-tuning
print("Starting fine-tuning for Anupama Pandey...")
trainer.train()

# Save for mobile conversion
model.save_pretrained("anupama_model")
tokenizer.save_pretrained("anupama_model")
print("Model saved successfully!")

# Test generation
prompt = "Anupama Pandey's birthday message:"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    inputs.input_ids,
    max_length=150,
    temperature=0.9,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Starting fine-tuning for Anupama Pandey...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model saved successfully!
Anupama Pandey's birthday message:
She was born on December 4th and we celebrate her birthday with great enthusiasm.
She works at the Civil Court where she serves with dedication and integrity.
With a Master's degree in Ancient History, she has deep knowledge about historical civilizations.
Her green thumb makes our garden flourish with roses, marigolds and tulips.
In the kitchen, she creates magic - her biryani and gulab jamun are legendary.
In the kitchen, she creates magic - her biryani and gulab jamun are legendary.
In the kitchen, she creates magic - her biryani gulabun are legendary.
In the kitchen, she creates magic - her biry


In [ ]:
prompt = "when is her birthday?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    inputs.input_ids,
    max_length=70,
    temperature=0.9,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
)

print(complete_sentences(tokenizer.decode(outputs[0], skip_special_tokens=True),prompt))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



She was born on December 4th and we celebrate her birthday with great enthusiasm.


In [ ]:
def complete_sentences(text, prompt):
    """
    Remove prompt from beginning and ensure complete sentence ending
    Args:
        text: Generated text (may contain prompt)
        prompt: The input prompt to remove
    Returns:
        Cleaned text without prompt and with proper ending
    """
    # First remove the prompt from beginning if it exists
    if prompt and text.startswith(prompt):
        text = text[len(prompt):].lstrip(" :")  # Also remove any trailing : or space

    # Then ensure proper sentence ending
    last_punct = max(
        text.find('.'),
        text.find('!'),
        text.find('?')
    )

    if last_punct == -1:
        # No punctuation found - add period if needed
        return text if text.endswith(('.','!','?')) else text + '.'
    else:
        # Truncate after last punctuation
        return text[:last_punct+1]

In [ ]:
    !zip -r anumpama.zip ./anupama_model/

  adding: anupama_model/ (stored 0%)
  adding: anupama_model/checkpoint-15/ (stored 0%)
  adding: anupama_model/checkpoint-15/training_args.bin (deflated 52%)
  adding: anupama_model/checkpoint-15/config.json (deflated 51%)
  adding: anupama_model/checkpoint-15/model.safetensors (deflated 7%)
  adding: anupama_model/checkpoint-15/tokenizer_config.json (deflated 56%)
  adding: anupama_model/checkpoint-15/generation_config.json (deflated 24%)
  adding: anupama_model/checkpoint-15/vocab.json (deflated 68%)
  adding: anupama_model/checkpoint-15/optimizer.pt (deflated 7%)
  adding: anupama_model/checkpoint-15/trainer_state.json (deflated 56%)
  adding: anupama_model/checkpoint-15/scheduler.pt (deflated 56%)
  adding: anupama_model/checkpoint-15/special_tokens_map.json (deflated 74%)
  adding: anupama_model/checkpoint-15/merges.txt (deflated 53%)
  adding: anupama_model/checkpoint-15/rng_state.pth (deflated 24%)
  adding: anupama_model/config.json (deflated 51%)
  adding: anupama_model/model

In [ ]:
!zip -r anupama.zip ./anupama_model/

  adding: anupama_model/ (stored 0%)
  adding: anupama_model/checkpoint-15/ (stored 0%)
  adding: anupama_model/checkpoint-15/training_args.bin (deflated 52%)
  adding: anupama_model/checkpoint-15/config.json (deflated 51%)
  adding: anupama_model/checkpoint-15/model.safetensors (deflated 7%)
  adding: anupama_model/checkpoint-15/tokenizer_config.json (deflated 56%)
  adding: anupama_model/checkpoint-15/generation_config.json (deflated 24%)
  adding: anupama_model/checkpoint-15/vocab.json (deflated 68%)
  adding: anupama_model/checkpoint-15/optimizer.pt (deflated 7%)
  adding: anupama_model/checkpoint-15/trainer_state.json (deflated 56%)
  adding: anupama_model/checkpoint-15/scheduler.pt (deflated 56%)
  adding: anupama_model/checkpoint-15/special_tokens_map.json (deflated 74%)
  adding: anupama_model/checkpoint-15/merges.txt (deflated 53%)
  adding: anupama_model/checkpoint-15/rng_state.pth (deflated 24%)
  adding: anupama_model/config.json (deflated 51%)
  adding: anupama_model/model

In [ ]:
!pip install gradio transformers torch

import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load your fine-tuned model
model_path = "./anupama_model"  # Update if different
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Improved response cleaning function
def complete_sentences(text, prompt):
    """
    Remove prompt from beginning and ensure complete sentence ending
    Args:
        text: Generated text (may contain prompt)
        prompt: The input prompt to remove
    Returns:
        Cleaned text without prompt and with proper ending
    """
    # First remove the prompt from beginning
    if prompt and text.startswith(prompt):
        text = text[len(prompt):].lstrip(" :")  # Remove prompt and any trailing spaces/colons

    # Find last punctuation (using rfind for last occurrence)
    last_punct = max(
        text.rfind('.'),  # Note: rfind() not find()
        text.rfind('!'),
        text.rfind('?')
    )

    if last_punct == -1:
        # No punctuation found - add period if needed
        return text if text.endswith(('.','!','?')) else text + '.'
    else:
        # Truncate after last punctuation
        return text[:last_punct+1]

# Function to generate responses
def generate_response(message, history):
    """Generate response using fine-tuned model"""
    # Build conversation context
    context = "You're Anupama AI, a helpful assistant based on Anupama Pandey's knowledge.\n"
    for human, assistant in history:
        context += f"Human: {human}\nAnupama AI: {assistant}\n"
    context += f"Human: {message}\nAnupama AI:"

    # Tokenize input
    inputs = tokenizer.encode(context, return_tensors="pt").to(device)
    input_length = inputs.shape[1]

    # Generate response
    outputs = model.generate(
        inputs,
        max_length=input_length + 200,  # Generate up to 200 new tokens
        temperature=0.7,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.1,
        num_beams=3,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        do_sample=True
    )

    # Extract and clean response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response[len(context):]  # Remove the context
    clean_response = complete_sentences(response, message)

    return clean_response.strip()

# Create ChatGPT-like UI
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# <center>🧠 Anupama AI Assistant</center>")
    gr.Markdown("### <center>Powered by your personalized GPT-2 model</center>")

    with gr.Row():
        with gr.Column(scale=3):
            # Add some visual elements
            gr.Image("https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/ChatGPT_logo.svg/240px-ChatGPT_logo.svg.png",
                     height=100, label="AI Assistant", show_label=True)

            gr.Markdown("""
            **About Anupama**:
            - From Lucknow, Uttar Pradesh
            - Works at Civil Court
            - MA in Ancient History
            - Loves gardening, cooking & Instagram reels
            - Birthday: December 4th
            """)

        with gr.Column(scale=7):
            # Chat interface
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(label="Your Message")
            clear = gr.Button("Clear Chat")

    # Event handling
    def respond(message, chat_history):
        bot_message = generate_response(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

# Run the app
app.launch(share=True)

/tmp/ipython-input-42-1981475897.py:98: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://14d9e83cb9adea1fc5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.6/725.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 8.0 MB/s eta 0:00:00
cp: './anupama_model/checkpoint-15' and './anupama_model/checkpoint-15' are the same file
cp: './anupama_model/config.json' and './anupama_model/config.json' are the same file
cp: './anupama_model/generation_config.json' and './anupama_model/generation_config.json' are the same file
cp: './anupama_model/merges.txt' and './anupama_model/merges.txt' are the same file
cp: './anupama_model/model.safetensors' and './anupama_model/model.safetensors' are the same file
cp: './anupama_model/special_tokens_map.json' and './anupama_model/special_tokens_map.json' are the same file
cp: './anupama_model/tokenizer_config.json' and './anupama_model/tokenizer_config.json' are the same f

UsageError: Line magic function `%%writefile` not found.


In [ ]:
# Step 1: Install required packages
!pip install pyinstaller transformers torch

# Step 2: Create the app.py file
with open("app.py", "w", encoding="utf-8") as f:
    f.write("""import tkinter as tk
from tkinter import scrolledtext, ttk, font
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import threading
import torch
import os

class AnupamaAIApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Anupama AI Assistant")
        self.root.geometry("800x600")
        self.root.configure(bg="#f0f2f5")

        # Set icon (optional)
        try:
            self.root.iconbitmap("icon.ico")
        except:
            pass

        # Load model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = GPT2Tokenizer.from_pretrained("anupama_model")
        self.model = GPT2LMHeadModel.from_pretrained("anupama_model").to(self.device)

        # Create UI
        self.create_widgets()

    def create_widgets(self):
        # Header
        header = ttk.Frame(self.root, style="Header.TFrame")
        header.pack(fill=tk.X, padx=10, pady=10)

        title = ttk.Label(header, text="🧠 Anupama AI Assistant",
                         font=("Arial", 16, "bold"),
                         style="Header.TLabel")
        title.pack(pady=5)

        subtitle = ttk.Label(header, text="Personal Assistant Trained on Anupama Pandey's Knowledge",
                           style="Subtitle.TLabel")
        subtitle.pack(pady=2)

        # Chat history
        self.chat_history = scrolledtext.ScrolledText(
            self.root,
            wrap=tk.WORD,
            font=("Arial", 11),
            bg="white",
            padx=10,
            pady=10
        )
        self.chat_history.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        self.chat_history.configure(state=tk.DISABLED)

        # Input area
        input_frame = ttk.Frame(self.root)
        input_frame.pack(fill=tk.X, padx=10, pady=10)

        self.user_input = ttk.Entry(
            input_frame,
            font=("Arial", 11),
            width=50
        )
        self.user_input.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        self.user_input.bind("<Return>", self.on_enter)

        send_button = ttk.Button(
            input_frame,
            text="Send",
            command=self.generate_response
        )
        send_button.pack(side=tk.RIGHT)

        # Configure styles
        self.configure_styles()

        # Add welcome message
        self.add_message("Anupama AI", "Hello! I'm your personal AI assistant trained on Anupama Pandey's knowledge. You can ask me about her work, hobbies, or birthday!")

    def configure_styles(self):
        style = ttk.Style()
        style.configure("Header.TFrame", background="#4f46e5")
        style.configure("Header.TLabel", background="#4f46e5", foreground="white")
        style.configure("Subtitle.TLabel", background="#4f46e5", foreground="#e0e7ff")
        style.configure("TButton", padding=6, font=("Arial", 10))
        style.configure("TEntry", padding=8)

        # Configure text tags
        self.chat_history.tag_configure("sender", foreground="#1e40af", font=("Arial", 11, "bold"))
        self.chat_history.tag_configure("ai", foreground="#7e22ce", font=("Arial", 11, "bold"))

    def add_message(self, sender, message):
        self.chat_history.configure(state=tk.NORMAL)
        tag = "sender" if sender == "You" else "ai"
        self.chat_history.insert(tk.END, f"{sender}: ", tag)
        self.chat_history.insert(tk.END, f"{message}\n\n")
        self.chat_history.configure(state=tk.DISABLED)
        self.chat_history.see(tk.END)

    def on_enter(self, event):
        self.generate_response()

    def generate_response(self):
        user_text = self.user_input.get().strip()
        if not user_text:
            return

        self.add_message("You", user_text)
        self.user_input.delete(0, tk.END)

        # Start generation in separate thread
        threading.Thread(target=self._generate_response, args=(user_text,), daemon=True).start()

    def _generate_response(self, user_text):
        # Add "thinking" message
        self.add_message("Anupama AI", "Thinking...")

        try:
            # Prepare input
            prompt = f"Human: {user_text}\\nAnupama AI:"
            inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)

            # Generate response
            outputs = self.model.generate(
                inputs,
                max_length=200,
                temperature=0.7,
                top_k=40,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id,
                num_return_sequences=1
            )

            # Decode and clean response
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = full_response[len(prompt):].strip()

            # Remove trailing incomplete sentences
            last_punct = max(response.rfind('.'), response.rfind('!'), response.rfind('?'))
            if last_punct != -1:
                response = response[:last_punct+1]

            # Remove "thinking" message and add actual response
            self.chat_history.configure(state=tk.NORMAL)
            self.chat_history.delete("end-3l linestart", "end")
            self.chat_history.configure(state=tk.DISABLED)

            self.add_message("Anupama AI", response)

        except Exception as e:
            self.add_message("Anupama AI", f"Sorry, I encountered an error: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = AnupamaAIApp(root)
    root.mainloop()
""")

# Step 3: Create the build script for Windows
with open("build.bat", "w") as f:
    f.write("""@echo off
echo Installing required packages...
pip install pyinstaller transformers torch

echo Building Anupama AI Assistant...
pyinstaller --onefile ^
    --add-data "anupama_model;anupama_model" ^
    --windowed ^
    --name "Anupama_AI_Assistant" ^
    --icon=icon.ico ^
    app.py

echo Build complete! Check dist/ folder for the executable.
echo If you don't have an icon, remove the --icon=icon.ico flag
pause
""")